# Connected Component Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import skimage.color
import skimage.filters
import skimage.measure

In [ ]:
image = skimage.io.imread("junk.jpg")
skimage.io.imshow(image)
plt.show()

In [ ]:
gray_image = skimage.color.rgb2gray(image)

sigma = 2.0
blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)

t = 0.9
binary_mask = blurred_image < t
skimage.io.imshow(binary_mask)
plt.show()

## Connected Component Analysis

In [ ]:
connectivity = 2
labeled_image, count = skimage.measure.label(binary_mask, connectivity=connectivity, return_num=True)
skimage.io.imshow(labeled_image)
plt.show()

In [ ]:
colored_label_image = skimage.color.label2rgb(labeled_image, bg_label=0)
skimage.io.imshow(colored_label_image)
plt.axis('off')
plt.savefig("../../fig/09-labeled-objects.png",dpi=150)
plt.show()

### How many objects are in that image

In [ ]:
def connected_components(filename, sigma=1.0, t=0.5, connectivity=2):
    image = skimage.io.imread(filename)
    gray_image = skimage.color.rgb2gray(image)
    blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < t
    labeled_image, count = skimage.measure.label(binary_mask, connectivity=connectivity, return_num=True)
    return labeled_image, count

labeled_image, count = connected_components("junk.jpg", sigma=2.0, t=0.9, connectivity=2)
skimage.io.imshow(labeled_image)

In [ ]:
print("Found", count, "objects in the image.")

In [ ]:
num_objects = np.max(labeled_image)
print("Found", num_objects, "objects in the image.")

In [ ]:
skimage.io.imshow(colored_label_image)
plt.xlim(0,200)
plt.ylim(1700,1300)
plt.show()

## Morphometrics - Describe object features with numbers

### Plot a histogram of the object area distribution

In [ ]:
# compute object features and extrac object areas
object_features = skimage.measure.regionprops(labeled_image)
object_areas = [objf["area"] for objf in object_features]
plt.hist(object_areas)
plt.savefig("../../fig/07-areas-histogram.png",dpi=150)
object_areas

### Filter objects by area

In [ ]:
min_area = 200
large_objects = [ area for area in object_areas if area > min_area ]
plt.hist(large_objects)
large_objects

In [ ]:
object_areas = np.array(object_areas)
np.count_nonzero(object_areas > min_area)

In [ ]:
object_labels = np.array([objf["label"] for objf in object_features])
small_object_labels = object_labels[np.where(object_areas < min_area)]
list(small_object_labels)

In [ ]:
def enhanced_connected_components(filename, sigma=1.0, t=0.5, connectivity=2, min_area=0):
    image = skimage.io.imread(filename)
    gray_image = skimage.color.rgb2gray(image)
    blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < t
    labeled_image, count = skimage.measure.label(binary_mask, connectivity=connectivity, return_num=True)
    object_features = skimage.measure.regionprops(labeled_image)
    object_areas = np.array([objf["area"] for objf in object_features])
    large_count = np.count_nonzero(object_areas > min_area)
    object_labels = np.array([objf["label"] for objf in object_features])
    small_object_labels = object_labels[np.where(object_areas < min_area)]
    labeled_image[np.isin(labeled_image,small_object_labels)] = 0
    return labeled_image, large_count

labeled_image, count = enhanced_connected_components("junk.jpg", sigma=2.0, t=0.9, connectivity=2, min_area=200)
colored_label_image = skimage.color.label2rgb(labeled_image, bg_label=0)
skimage.io.imshow(colored_label_image)
plt.axis('off')
plt.savefig("../../fig/09-filtered-objects.png",dpi=150)
print("Found", count, "objects in the image.")

In [ ]:
colored_label_image = np.insert(0,1,object_areas)[labeled_image]
fig = skimage.io.imshow(colored_label_image)
fig.figure.get_axes()[0].axis('off')
fig.figure.get_axes()[1].set_title('area')
plt.savefig("../../fig/09-objects-colored-by-area.png",dpi=150)